In [3]:
# Importing the necessary package 
# Make sure to pip install it in Anaconda, if you don't have it
# https://finance.yahoo.com/quote/AC.TO?p=AC.TO&.tsrc=fin-srch


try:
    import yfinance 
except:
    ! pip install yfinance

In [4]:
"""
A simple Stress test on Pandemic reovery Index using ARIMA 
Author : Babak.Ea
Date   : 2020-12-09
Target : ARIMA stress test on Recovery curves 

"""

###############################

from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import datetime
plt.style.use('ggplot')
import pandas as pd
from pandas.io.json import json_normalize 
import numpy as np
import statsmodels
import datetime
import urllib, json
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
import sys
import warnings
import ipywidgets as widgets
from IPython.display import display

import statsmodels.graphics.tsaplots as sgt 
import statsmodels.tsa.stattools as sts 
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()

from math import sqrt
from sklearn.metrics import mean_squared_error



if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [5]:
# Ignoring warning messages
import warnings 
warnings.filterwarnings("ignore")

In [6]:
# Using the .download() method to get our data

raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI AC.TO", #The time series we are interested in - (in our case, these are the S&P, FTSE, NIKKEI and DAX)
                              start = "1994-01-07", #The starting date of our data set
                              end = "2020-12-05", #The ending date of our data set (at the time of upload, this is the current date)
                              interval = "1d", #The distance in time between two recorded observations. Since we're using daily closing prices, we set it equal to "1d", which indicates 1 day. 
                              group_by = 'ticker', #The way we want to group the scraped data. Usually we want it to be "ticker", so that we have all the information about a time series in 1 variable.
                              auto_adjust = True, #Automatically adjuss the closing prices for each period. 
                              treads = True) #Whether to use threads for mass downloading. 

[*********************100%***********************]  5 of 5 completed


In [7]:
raw_data.to_csv("Yahoo_stocks.csv")
raw_data.fillna(0)

AC.TO                                              \
                 Open       High        Low      Close     Volume   
Date                                                                
1994-01-07   0.000000   0.000000   0.000000   0.000000        0.0   
1994-01-10   0.000000   0.000000   0.000000   0.000000        0.0   
1994-01-11   0.000000   0.000000   0.000000   0.000000        0.0   
1994-01-12   0.000000   0.000000   0.000000   0.000000        0.0   
1994-01-13   0.000000   0.000000   0.000000   0.000000        0.0   
...               ...        ...        ...        ...        ...   
2020-11-30  25.500000  25.600000  23.950001  24.700001  8967854.0   
2020-12-01  25.059999  25.320000  24.719999  25.110001  6342087.0   
2020-12-02  25.170000  26.059999  24.990000  25.889999  7100811.0   
2020-12-03  26.500000  27.500000  26.230000  27.250000  9729532.0   
2020-12-04  27.950001  28.240000  27.469999  27.500000  8084565.0   

                  ^FTSE                                                       \
                   Open         High          Low        Close        Volume   
Date                                                                           
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000  0.000000e+00   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098  0.000000e+00   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049  0.000000e+00   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000  0.000000e+00   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000  0.000000e+00   
...                 ...          ...          ...          ...           ...   
2020-11-30  6367.600098  6404.000000  6266.200195  6266.200195  1.355075e+09   
2020-12-01  6266.200195  6400.200195  6263.600098  6384.700195  8.398583e+08   
2020-12-02  6384.700195  6468.600098  6367.500000  6463.399902  9.368428e+08   
2020-12-03  6463.399902  6491.500000  6430.299805  6490.299805  9.432145e+08   
2020-12-04  6490.299805  6559.200195  6489.799805  6550.200195  1.073111e+09   

            ...        ^GDAXI                                            \
            ...          Open          High           Low         Close   
Date        ...                                                           
1994-01-07  ...   2218.959961   2227.639893   2201.820068   2224.949951   
1994-01-10  ...   2231.840088   2238.010010   2222.000000   2225.000000   
1994-01-11  ...   2225.429932   2235.610107   2225.179932   2228.100098   
1994-01-12  ...   2227.120117   2227.790039   2182.060059   2182.060059   
1994-01-13  ...   2171.500000   2183.709961   2134.100098   2142.370117   
...         ...           ...           ...           ...           ...   
2020-11-30  ...  13270.509766  13445.110352  13256.759766  13291.160156   
2020-12-01  ...  13371.660156  13434.299805  13334.990234  13382.299805   
2020-12-02  ...  13341.910156  13359.269531  13272.469727  13313.240234   
2020-12-03  ...  13304.830078  13307.540039  13223.929688  13252.860352   
2020-12-04  ...  13230.610352  13315.370117  13225.080078  13298.959961   

                                ^N225                              \
                 Volume          Open          High           Low   
Date                                                                
1994-01-07          0.0  17842.980469  18131.410156  17787.480469   
1994-01-10          0.0  18186.519531  18567.060547  18186.519531   
1994-01-11          0.0  18481.849609  18671.669922  18373.039062   
1994-01-12          0.0  18447.339844  18807.080078  18301.929688   
1994-01-13          0.0  18770.380859  18823.380859  18548.750000   
...                 ...           ...           ...           ...   
2020-11-30  107366400.0  26830.099609  26834.199219  26405.830078   
2020-12-01   66675500.0  26624.199219  26852.160156  26618.390625   
2020-12-02   63822100.0  26884.990234  26889.900391  26695.279297   
2020-12-03   64310300.0  26740.300781  26868.0898

In [8]:
# Creating a back up copy in case we remove/alter elements of the data by mistake
df_comp = raw_data.copy()

In [9]:
# Adding new columns to the data set
df_comp['spx'] = df_comp['^GSPC'].Close
df_comp['dax'] = df_comp['^GDAXI'].Close
df_comp['ftse'] = df_comp['^FTSE'].Close
df_comp['nikkei'] = df_comp['^N225'].Close

In [10]:
df_comp = df_comp.iloc[1:] # Removing the first elements, since we always start 1 period before the first, due to time zone differences of closing prices
del df_comp['^N225']  # Removing the original tickers of the data set
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b') # Setting the frequency of the data
df_comp=df_comp.fillna(method='ffill') # Filling any missing values

In [11]:
print (df_comp.head()) # Displaying the first 5 elements to make sure the data was scrapped correctly
print (df_comp.tail()) # Making sure of the last day we're including in the series

           AC.TO                               spx          dax         ftse  \
            Open High Low Close Volume                                         
Date                                                                           
1994-01-10   NaN  NaN NaN   NaN    NaN  475.269989  2225.000000  3440.600098   
1994-01-11   NaN  NaN NaN   NaN    NaN  474.130005  2228.100098  3413.800049   
1994-01-12   NaN  NaN NaN   NaN    NaN  474.170013  2182.060059  3372.000000   
1994-01-13   NaN  NaN NaN   NaN    NaN  472.470001  2142.370117  3360.000000   
1994-01-14   NaN  NaN NaN   NaN    NaN  474.910004  2151.050049  3400.600098   

                  nikkei  
                          
Date                      
1994-01-10  18443.439453  
1994-01-11  18485.250000  
1994-01-12  18793.880859  
1994-01-13  18577.259766  
1994-01-14  18973.699219  
                AC.TO                                              \
                 Open       High        Low      Close     Volume   
Date 